# Text generator based on RNN

### Import libraries

In [1]:
import tensorflow as tf
import numpy as np

### Configurations

In [2]:
vocab = (" $%'()+,-./0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ"
            "\\^_abcdefghijklmnopqrstuvwxyz{|}")
TestText="Hello World. "*2
graphPath=r".\graphs"
num_step=len(TestText)
state_size=100#Make it simple
batchSize=1
def Encode(string,name=None):
    tensor=[vocab.find(ch)+1 for ch in string]
    tensor = tf.one_hot(tensor, depth=len(vocab)+1, on_value=1.0, off_value=0.0, axis=-1, dtype=tf.float32, name=name)
    with tf.Session() as sess:
        nparray=tensor.eval()
    return nparray
def Decode(nparray):
    return "".join([vocab[index-1] if index>0 else "[INVALID]" for index in np.argmax(nparray,axis=1)])
data=Encode(TestText)

### Create graph

In [3]:
with tf.variable_scope("RNN") as modelScope:
    with tf.name_scope("structure"):
        inputs=tf.placeholder(dtype=tf.float32,shape=[None,len(vocab)+1])
        cell=tf.contrib.rnn.GRUCell(num_units=state_size)
        w=tf.get_variable(name="Weight_s_o",shape=[state_size,len(vocab)],initializer=tf.truncated_normal_initializer())
        b=tf.get_variable(name="Bias_s_o",shape=[1,len(vocab)],initializer=tf.truncated_normal_initializer())
        initState=cell.zero_state(batch_size=batchSize,dtype=tf.float32)
        state=initState
        outputs=[]
    for i in range(num_step):
        if(i>0):
            modelScope.reuse_variables()
        output,state=cell(inputs=tf.reshape(tensor=inputs[i],shape=[1,-1]),state=state)
        output=tf.nn.softmax(tf.add(tf.matmul(output,w),b,name="Output"))
        outputs.append(output)
    with tf.name_scope("summary") as scope:
        summary_op=tf.summary.merge_all()   

In [4]:
with tf.Session() as sess:
    writer=tf.summary.FileWriter(logdir=graphPath,graph=sess.graph)
    sess.run(tf.global_variables_initializer())
    result=sess.run(outputs,{inputs:data})
    result=np.concatenate(result,axis=0)
    print(result)
print(Decode(result))

[[ 0.00230916  0.00361426  0.00704901 ...,  0.00358786  0.0043457
   0.00752503]
 [ 0.00196056  0.00483805  0.00592424 ...,  0.00376358  0.00407425
   0.00767895]
 [ 0.00240414  0.00434853  0.00461183 ...,  0.00495821  0.00419408
   0.00727658]
 ..., 
 [ 0.00209293  0.00267348  0.00534597 ...,  0.00848178  0.00396167
   0.00813794]
 [ 0.0019422   0.0025199   0.00663691 ...,  0.00669845  0.00455267
   0.00805597]
 [ 0.00161771  0.00314868  0.00667456 ...,  0.00552558  0.00545918
   0.00791381]]
%%%%KKKKKKKKK%%%mKKKKKKKKK
